- Shapley Value란, 게임이론을 바탕으로 Game에서 각 Player의 기여분을 계산하는 방법임.
- 하나의 feature에 대한 중요도를 얻기 위해 다양한 feature의 조합을 구성하고, 해당 feature의 유무에 따른 평균적인 변화를 통해 얻은 값임.
- 따라서 Shapley Value는 전체 성과(판단)을 창출하는 데 각 feature가 얼마나 공헌했는지 수치로 표현할 수 있음.

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import warnings
import pickle

import shap

warnings.filterwarnings('ignore')

In [ ]:
# Setting parameter
rand_seed = 5

# loading data & model
df = pd.read_csv("data/selected_tree.csv" , sep = ",")

X = df.drop('label', axis=1)  # Features
Y = df['label']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=rand_seed, stratify=Y)


print(f'Train: {X_train.shape} | Test: {X_test.shape}')


with open("./model_rf.pkl", "rb") as f:
    model = pickle.load(f)


In [ ]:
# TreeExplainer 객체 생성
exp = shap.TreeExplainer(model)

# SHAP값 산출
sv_test = exp.shap_values(X_test)
sv_test = sv_test[:,:,1] #클래스 1에 속할 확률에 대한 SHAP값

# 결과 확인
print(X_test.shape, sv_test.shape) # 데이터 세트와 SHAP 값의 형태
print(sv_test[0]) # 첫 번째 샘플에 대한 SHAP 값

In [ ]:
# 이름 수정
feature_names = df.drop('label', axis=1).columns.to_list()
feature_names = [n[n.index('_')+1:] for n in feature_names]
print(feature_names)

In [ ]:
exp.shap_values(X_test).shape
print(X_test.iloc[0])

In [ ]:
"""
# 개별데이터에 대한 특징량을 시각화
    그래프의 수평선은 각 특징의 영향을 의미.
    빨간색 선: Feature가 해당 클래스로 예측함을 증가시키는 방향으로 영향을 미침을 의미.
    파란색 선: Feature가 해당 클래스로 예측함을 감소시키는 방향으로 영향을 미침을 의미.
    선의 길이: 영향의 크기와 비례함.=> 긴 선일수록 더 큰 영향.
    모든 선의 영향의 합 = 모델의 최종 예측값.

# exp.expected_value[1]는 이진 분류 문제에서 클래스 1에 대한 기대값을 나타냄
# sv_test[n]: 개별 데이터의 모든 특징의 SHAP 값을 나타냄
# test_x[n]: 개별 데이터의 대한 특징 값을 나타냄

"""

sample_num = 0

shap.force_plot(base_value=exp.expected_value[1], shap_values=sv_test[sample_num], features=X_test.iloc[sample_num],
                feature_names=feature_names, matplotlib=True, figsize=(20, 5))
plt.tight_layout() #레이아웃을 조정하여 글자가 겹치지 않게 함
plt.show()


In [ ]:
"""
    그래프의 수평선은 기본 예측값에서 시작하여 각 특징의 영향을 순차적으로 더해 나감.
    빨간색 선은 Feature가 예측을 증가시키는 영향을 미친다는 것을 의미.
    파란색 선은 Feature가 예측을 감소시키는 영향을 미친다는 것을 의미.
    선의 기울기는 영향의 크기를 의미=> 가파른 선일수록 더 큰 영향.
    최종 예측값은 그래프의 오른쪽 끝에 도달한 값.
"""
shap.decision_plot(
    base_value=exp.expected_value[1], shap_values=sv_test[sample_num], features=X_test.iloc[sample_num], feature_names=feature_names)

In [ ]:
# 전체 데이터 세트에 대한 특징량의 영향 시각화
"""
    그래프의 각 행은 하나의 특징을 의미.
    Feature의 중요도는 그래프에서 위에서부터 아래로 정렬=> 위쪽에 있는 특징일수록 더 중요도 높은 특징.
    각 점은 특정 데이터 포인트에서 해당 Feature의 SHAP 값.
    색상은 Feature의 실제 값을 의미. 높은 값은 빨간색으로, 낮은 값은 파란색으로 표시.
    점의 수평 위치는 해당 Feature가 예측에 어떤 영향을 미치는지 의미. 
    => 오른쪽으로 갈수록 예측을 증가시키는 영향을 미치며, 왼쪽으로 갈수록 예측을 감소시키는 영향.
"""
## SAHP은 모델의 국소적인 설명에 특화된 기술이지만, 
## summary_plot 함수를 사용 시 모델의 전체적인 설명을 파악 가능
shap.summary_plot(shap_values=sv_test, features=X_test,
                  feature_names=feature_names)